In [1]:
import glob
import json

import numpy
import tensorflow as tf

import tools_keras

16

In [19]:
with open(glob.glob("../urbangrammar_samba/spatial_signatures/ai/gb_64_shuffled/json/*")[0]) as f:
    meta = json.load(f)

In [20]:
model = tf.keras.models.load_model(glob.glob("../urbangrammar_samba/spatial_signatures/ai/gb_64_shuffled/model/*")[0])

In [21]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    "../chips_gb/64_shuffled_balanced/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False,
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(
    "../chips_gb/64_shuffled_balanced/validation",
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)
secret_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
secret_generator = validation_datagen.flow_from_directory(
    "../chips_gb/64_shuffled_balanced/secret",
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

subsets = {
    "train": train_generator,
    "val": validation_generator,
    "secret": secret_generator,
}

Found 125450 images belonging to 5 classes.
Found 26750 images belonging to 5 classes.
Found 24017 images belonging to 5 classes.


In [22]:
for subset in subsets:
    dataset = subsets[subset]
    y_pred_probs = numpy.load(f"../urbangrammar_samba/spatial_signatures/ai/gb_64_shuffled/pred/efficientnet_pooling_256_5_{subset}.npy")
    y_pred = numpy.argmax(y_pred_probs, axis=1)
    y = dataset.labels
    top_prob, wc_accuracy, wc_top_prob = tools_keras.within_class_metrics(
            y, y_pred, y_pred_probs
        )
    # Prob for top class
    meta[f"perf_avg_prob_top_{subset}"] = top_prob
    # Within-class accuracy
    meta[f"perf_within_class_accuracy_{subset}"] = wc_accuracy
    # Within-class avg. prob for top class
    meta[f"perf_within_class_avg_prob_top_{subset}"] = wc_top_prob

In [23]:
with open(glob.glob("../urbangrammar_samba/spatial_signatures/ai/gb_64_shuffled/json/*")[0], "w") as f:
    f.write(json.dumps(meta, indent=4, cls=tools_keras.NumpyEncoder).replace("NaN", "null"))